In [174]:
import json
import pandas as pd
import requests
import wbgapi as wb
from functions import ember_fetch_data

# country-codes: https://www.iban.com/country-codes
countries = ('AFG', 'ALB', 'DZA', 'ASM', 'AGO', 'ATG', 'ARG', 'ARM', 'ABW',
       'AUS', 'AUT', 'AZE', 'BHS', 'BHR', 'BGD', 'BRB', 'BLR', 'BEL',
       'BLZ', 'BEN', 'BMU', 'BTN', 'BOL', 'BIH', 'BWA', 'BRA', 'VGB',
       'BRN', 'BGR', 'BFA', 'BDI', 'KHM', 'CMR', 'CAN', 'CPV', 'CYM',
       'CAF', 'TCD', 'CHL', 'CHN', 'COL', 'COM', 'COG', 'COK', 'CRI',
       'CIV', 'HRV', 'CUB', 'CYP', 'CZE', 'COD', 'DNK', 'DJI', 'DMA',
       'DOM', 'ECU', 'EGY', 'SLV', 'GNQ', 'ERI', 'EST', 'SWZ', 'ETH',
       'FRO', 'FLK', 'FJI', 'FIN', 'FRA', 'GUF', 'PYF', 'GAB', 'GMB',
       'GEO', 'DEU', 'GHA', 'GIB', 'GRC', 'GRL', 'GRD', 'GLP', 'GUM',
       'GTM', 'GIN', 'GNB', 'GUY', 'HTI', 'HND', 'HKG', 'HUN', 'ISL',
       'IND', 'IDN', 'IRN', 'IRQ', 'IRL', 'ISR', 'ITA', 'JAM', 'JPN',
       'JOR', 'KAZ', 'KEN', 'KIR', 'KWT', 'KGZ', 'LAO', 'LVA', 'LBN',
       'LSO', 'LBR', 'LBY', 'LTU', 'LUX', 'MAC', 'MDG', 'MWI', 'MYS',
       'MDV', 'MLI', 'MLT', 'MTQ', 'MRT', 'MUS', 'MEX', 'MDA', 'MNG',
       'MNE', 'MSR', 'MAR', 'MOZ', 'MMR', 'NAM', 'NRU', 'NPL', 'NLD',
       'NCL', 'NZL', 'NIC', 'NER', 'NGA', 'NIU', 'PRK', 'MKD', 'NOR',
       'OMN', 'PAK', 'PSE', 'PAN', 'PNG', 'PRY', 'PER', 'PHL', 'POL',
       'PRT', 'PRI', 'QAT', 'REU', 'ROU', 'RUS', 'RWA', 'SHN', 'KNA',
       'LCA', 'SPM', 'VCT', 'WSM', 'STP', 'SAU', 'SEN', 'SRB', 'SYC',
       'SLE', 'SGP', 'SVK', 'SVN', 'SLB', 'SOM', 'ZAF', 'KOR', 'SSD',
       'ESP', 'LKA', 'SDN', 'SUR', 'SWE', 'CHE', 'SYR', 'TWN', 'TJK',
       'TZA', 'THA', 'TLS', 'TGO', 'TON', 'TTO', 'TUN', 'TUR', 'TKM',
       'TCA', 'UGA', 'UKR', 'ARE', 'GBR', 'USA', 'VIR', 'URY', 'UZB',
       'VUT', 'VEN', 'VNM', 'ESH', 'YEM', 'ZMB', 'ZWE')

url1 = 'data/ren_ren_65-22/03_modern-renewable-prod.csv'
df_energy_prod = pd.read_csv(url1)

ImportError: cannot import name 'ember_fetch_data' from 'functions' (/Users/timniwo/Documents/Ironhack/final project/wattsup/functions.py)

In [140]:
# Dataframe Collection
df_names = ['df_energy_prod', 'df_energy_prod_world', 'df_energy_prod_regions', 'df_countries']

# Create empty DataFrames if they don't exist
for name in df_names:
    if name not in globals():
        globals()[name] = pd.DataFrame()

## API definition

In [143]:
# FastAPI by Ember
# https://api.ember-energy.org/v1/docs
base_url = 'https://api.ember-energy.org'
my_api_key = 'ddbb448b-f721-4fc7-9b52-68ad159bf482'

# dictionary with endpoints that may be necessary for the project.
EMBER_ENDPOINTS = {
    'generation_per_year': 'electricity-generation/yearly',
    'generation_per_month': 'electricity-generation/monthly',
    'demand_per_year': 'electricity-demand/yearly',
    'demand_per_month': 'electricity-demand/monthly'
}

In [145]:
# WBGAPI - World Bank Data
# !pip install wbgapi
# https://pypi.org/project/wbgapi/

# dictionary with extracted indicators that may be necessary for the project.
GDP_INDICATORS = {
    # Constant currency value indicators
    'GDP_constant_2015_USD': 'NY.GDP.MKTP.KD',
    'GDP_per_capita_constant_2015_USD': 'NY.GDP.PCAP.KD',
    'GDP_PPP_constant_2021_international_USD': 'NY.GDP.MKTP.PP.KD',
    'GDP_per_capita_PPP_constant_2021_international_USD': 'NY.GDP.PCAP.PP.KD',

    # Growth
    'GDP_growth_annual_percent': 'NY.GDP.MKTP.KD.ZG',

    # Current currency value indicators
    'GDP_current_USD': 'NY.GDP.MKTP.CD',
    'GDP_per_capita_current_USD': 'NY.GDP.PCAP.CD',
    'GDP_PPP_current_international_USD': 'NY.GDP.MKTP.PP.CD',
    'GDP_per_capita_PPP_current_international_USD': 'NY.GDP.PCAP.PP.CD',
}

## Generating Data

In [148]:
df_energy_prod = df_energy_prod.rename(columns={'Entity': 'country', 'Code': 'iso_code', 'Year': 'year', 'Electricity from wind (TWh)': 'wind', 'Electricity from hydro (TWh)': 'hydro', 'Electricity from solar (TWh)': 'solar', 'Other renewables including bioenergy (TWh)': 'other_inc_bio'})
# remove kosovo, udssr, regions and world & save regions and world in separate dfs
df_energy_prod = df_energy_prod.loc[df_energy_prod['iso_code'] != 'OWID_KOS']
df_energy_prod = df_energy_prod.loc[df_energy_prod['iso_code'] != 'OWID_USS']
df_energy_prod_world = df_energy_prod.loc[df_energy_prod['iso_code'] == 'OWID_WRL']
df_energy_prod = df_energy_prod.loc[df_energy_prod['iso_code'] != 'OWID_WRL']
df_energy_prod_regions = df_energy_prod[df_energy_prod['iso_code'].isna()]
df_energy_prod = df_energy_prod.loc[df_energy_prod['iso_code'].notna()]

In [150]:
# create countries dataframe
df_countries['name'] = df_energy_prod['country'].unique()
df_countries['iso_code'] = df_energy_prod['iso_code'].unique()

In [152]:
# map countries index on iso_code and remove redundant columns
#df_energy_prod['fk_country'] = df_energy_prod['Code'].map(lambda iso: df_countries[df_countries['iso_code'] == iso].index[0])
#df_energy_prod.drop(['Entity', 'Code'], axis=1, inplace=True)

In [ ]:
df_countries['iso_code'].unique()

In [ ]:
df_energy_prod

In [ ]:
df_energy_prod[df_energy_prod['iso_code']=='DEU']

In [ ]:
#df = wb.source.info()
#wb.search('gdp')
#wb.data.DataFrame(['NY.GDP.PCAP.CD', 'SP.POP.TOTL'], 'DEU', mrv=5)
#wb.data.Dataframe('selected_metric', 'USA')
#wb.economy.info(q='ger')
#for row in wb.data.fetch(selected_metric, 'USA'): # all years
    #print(row)

#wb.get_data(selected_metric, country=['USA', 'GBR'], date=('2010', '2011'))
wb.data.DataFrame(GDP_INDICATORS['GDP_growth_annual_percent'], countries, range(2021, 2025))


In [166]:
data = ember_fetch_data(EMBER_ENDPOINTS['generation_per_year'], countries, my_api_key, base_url)

#get_data(EMBER_ENDPOINTS['generation_per_year'], countries, my_api_key, 'output_data.json')

NameError: name 'ember_fetch_data' is not defined

In [ ]:
func.save_json(data, 'test.json')

In [ ]:
df_json_data = func.load_json('test.json')

In [ ]:
df_json_filtered = func.filter_data(df_json_data)
df_json_filtered

In [ ]:
df_json_agg = func.aggregate_bioenergy_other_renewables(df_json_filtered)
df_json_agg

In [ ]:
df_json_reshaped = func.reshape_data(df_json_agg)

In [ ]:
df_json_reshaped.head(50)

In [ ]:
url = 'http://api.worldbank.org/v2/country/%s/indicator/NY.GDP.PCAP.CD?format=json'
countries = ['DZA','AGO','ARG','AUS','AUT','BEL','BRA','CAN','CHL','CHN','COL','CYP', 'CZE','DNK','FIN','FRA','GEO','DEU', 'GRC','HUN','ISL','IND','IDN','IRL','ISR','ITA','JPN','KAZ','KWT','LBN','LIE','MYS','MEX','MCO','MAR','NPL','NLD', 'NZL','NGA','NOR','OMN','PER','PHL','POL','PRT','QAT','ROU','SGP','ZAF','ESP','SWE','CHE','TZA','THA','TUR','UKR', 'GBR','USA','VNM','ZWE']
my_values = []

for country in countries:
    data = requests.get(url % country).json()
    try:
        for d in data:inlineRefs{references='&#91;&#123;&quot;type&quot;&#58;&quot;inline_reference&quot;,&quot;start_index&quot;&#58;814,&quot;end_index&quot;&#58;817,&quot;number&quot;&#58;1,&quot;url&quot;&#58;&quot;https&#58;//deepnote.com/app/carlos-mendez/PYTHON-World-Bank-data-7d57b121-2bdf-4f13-b8d9-9520c9f8a7cc&quot;,&quot;favicon&quot;&#58;&quot;https&#58;//imgs.search.brave.com/8dmWbpa5EnrU1I0iyMuzw062ZJUziGPaoocSSmRgWlg/rs&#58;fit&#58;32&#58;32&#58;1&#58;0/g&#58;ce/aHR0cDovL2Zhdmlj/b25zLnNlYXJjaC5i/cmF2ZS5jb20vaWNv/bnMvNjZlM2JmNWIw/OGQzNmQ0NzUxMTIz/ODAyODczZjM5OGI1/YzhhYTNmZDY3MWVk/NzkwNjUzY2MxNTc5/ODUxZTY1MC9kZWVw/bm90ZS5jb20v&quot;,&quot;snippet&quot;&#58;&quot;Data&#32;apps&#32;for&#32;data&#32;scientists&#32;and&#32;data&#32;analysts…&quot;&#125;&#93;'}:
            my_values.append({
                'country': d['country']['value'],
                'date': d['date'],
                'value': d['value']
            })
    except Exception as err:
        print(f'[ERROR] country ==> {country} error ==> {err}')

df = pd.DataFrame(my_values).sort_values(['country', 'date'], ascending=True)